In [84]:
import pandas as pd
import numpy as np

In [85]:
data = pd.read_csv('BTC-USD.csv')

In [86]:
data['Date'] = pd.to_datetime(data['Date'])
data.dtypes

Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Volume              float64
dtype: object

In [87]:
data = data.set_index('Date')
data_freq = data.asfreq('D', method='ffill')
print(data.head())
print(data.index)

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2014-09-17  465.864014  468.174011  452.421997  457.334015  457.334015   
2014-09-18  456.859985  456.859985  413.104004  424.440002  424.440002   
2014-09-19  424.102997  427.834991  384.532013  394.795990  394.795990   
2014-09-20  394.673004  423.295990  389.882996  408.903992  408.903992   
2014-09-21  408.084991  412.425995  393.181000  398.821014  398.821014   

                Volume  
Date                    
2014-09-17  21056800.0  
2014-09-18  34483200.0  
2014-09-19  37919700.0  
2014-09-20  36863600.0  
2014-09-21  26580100.0  
DatetimeIndex(['2014-09-17', '2014-09-18', '2014-09-19', '2014-09-20',
               '2014-09-21', '2014-09-22', '2014-09-23', '2014-09-24',
               '2014-09-25', '2014-09-26',
               ...
               '2021-07-17', '2021-07-18', '2021-07-19', '2021-07-20',
               '2021-07-21', '2

In [88]:
forecast_out = int(1)
data['Prediction'] = data[['Adj Close']].shift(-forecast_out)

In [89]:
data.tail()

,Open,High,Low,Close,Adj Close,Volume,Prediction
Date,,,,,,,
2021-07-22,32138.873047,32576.400391,31745.298828,32313.105469,32313.105469,1.955523e+10,33581.550781
2021-07-23,32305.958984,33581.550781,32057.892578,33581.550781,33581.550781,2.255205e+10,34292.445313
2021-07-24,33593.730469,34490.390625,33424.859375,34292.445313,34292.445313,2.166471e+10,35350.187500
2021-07-25,34290.292969,35364.925781,33881.835938,35350.187500,35350.187500,2.085669e+10,39061.367188
2021-07-26,35287.312500,39061.367188,35287.312500,39061.367188,39061.367188,3.519676e+10,NaN


In [90]:
from sklearn import preprocessing
feature_cols = ['Adj Close']
X = data[feature_cols]
X = preprocessing.scale(X)
Pred_data = data[-forecast_out:]
X = X[:-forecast_out]
y = np.array(data['Prediction'])[:-forecast_out]
print(y)

[  424.440002   394.79599    408.903992 ... 34292.445313 35350.1875
 39061.367188]


In [91]:
print(X.shape)
print(y.shape)

(2504, 1)
(2504,)


In [92]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
logreg = LinearRegression()

In [93]:
scores = cross_val_score(logreg, X, y, cv=10, scoring='neg_mean_squared_error')
print(scores.mean())

nan


C:\Users\Wesley\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Wesley\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Wesley\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_base.py", line 518, in fit
    X, y = self._validate_data(X, y, accept_sparse=accept_sparse,
  File "C:\Users\Wesley\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py", line 433, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "C:\Users\Wesley\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py", line 63, in inner_f
    return f(*args, *